<a href="https://colab.research.google.com/github/smduarte/RC2021-labs/blob/main/aula5/lab5_cnss_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNSS (Computer Networks Simple Simulator)

CNSS (Computer Networks Simple Simulator) makes it possible to code and simulate simple networking protocols. The simulation results are deterministic and repeatable, making it easier to reproduce and interpret them.

In this lab we will first cover the basics of CNSS. Firstly, how and where to fetch the source code, how to compile the code and execute the simulator. Then we will learn about the basic components: **nodes** and **links** that will be used to simulate simple computer networks and networking protocols.

---

## Fetching CNSS

CNSS is written in Java 8. The source code is publicly available and hosted at [GitHub](https://github.com/jlegatheaux/cnss)

For those still unfamiliar with GitHub repositories, there are several ways
to access the source code.

You can [download a zip archive](https://github.com/jlegatheaux/cnss/archive/master.zip) of the entire repository;

You can also use [git](https://git-scm.com/) to clone the contents of the repository to a local directory, like so:

In [ ]:
%%bash

git clone https://github.com/jlegatheaux/cnss

This will create a directory ***cnss*** in the current directory with a local repository with the latest version.

You can refresh and update the local repository, by executing the following command in the repository folder:


In [ ]:
%%bash

git -C cnss pull

### Eclipse Users

CNSS can also be imported directly to Eclipse.

For version 2020-06, the procedure is as follows:
  * Copy the CNSS repository uri (https://github.com/jlegatheaux/cnss.git) into the clipboard;
  * Navigate: ```File > Import > Git```, choose ```Projects from Git```, then ```Next```
  * Choose ```Clone URI```, then ```Next``` twice.
  * If asked, only select the ```master``` branch and press ```Next``` until finished.

## Compiling CNSS manually

CNSS has no external dependencies. 

At the root of the repository, the source code can be compiled in the command line, like so:

In [ ]:
%%bash

mkdir cnss-classes
javac -d cnss-classes cnss/src/*/*/*.java

Note: the switch ```-d cnss-classes``` will place the resulting classes in the ```cnss-classes``` directory.

## Running CNSS

To run the simulator, a **plain text** configuration file is needed.

We can use an empty file as its argument just to try out the simulator.

In [ ]:
%%bash
echo > empty.config.txt

java -cp .:cnss-classes cnss.simulator.Simulator empty.config.txt

### Eclipse Users

Execute the [Simulator](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/Simulator.java) class, using the desired configuration file as its sole argument.

See [here](https://www.codejava.net/ides/eclipse/how-to-pass-arguments-when-running-a-java-program-in-eclipse) how to pass arguments to a Java program in Eclipse.

### Accessing the Google Colab Terminal 

Google Colab notebooks execute in a virtualized Linux environment. 

Executing the Python code below will provide a text console/terminal. This terminal provides limited shell access to the virtual machine where this notebook is running. It is a useful way to check the filesystem or run Unix commands interactively...

In [ ]:
#@title
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)
from IPython.core.display import HTML
HTML("""
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });
""")

---
# CNSS Overview

A simulator is piece of software that executes **actions** in response to **events**. To make the execution of a simulation **deterministic** and **repeatable**, the **ordering of events** is logical and **fixed**, and does not depend on the actual time taken to execute the actions, which are treated as **instantaneous**.

Given the computer networks context, CNSS simulates a network made of links that interconnect nodes. Nodes, using the network, can exchange messages (packets). In this context, an event in CNSS can be, for example, the *delivery* of a message at some node or the indication that a *timeout* has expired in another one, or yet the firing of a clock tick, ...

A more compreenhesive description of CNSS can be found [here](https://github.com/jlegatheaux/cnss).

## Nodes

A simulation will comprise of a number of nodes. Nodes can be used to simulate simple end-systems (application hosts) or other lower-level networking components, such as *packet switches*.

### Programming Application Nodes

The behavior of application nodes (or the application part of a node) is coded in a Java class that **implements** the [ApplicationAlgorithm](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/ApplicationAlgorithm.java) interface.

The CNSS codebase also provides a convenience class  [AbstractAplicationAlgorithm](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/lib/AbstractApplicationAlgorithm.java) that can be extended as needed. Using this class we avoid the need to provide implementions for interface methods that are not used. As a bare minimum,
we only need to provide a public no-args constructor and implement the **initialize** method of our node.

As an example, consider the following class:

In [ ]:
%%writefile MinimalNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class MinimalNode extends AbstractApplicationAlgorithm {

  public MinimalNode() {
      super(true, "minimal-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);

    super.log( now, "args: " + Arrays.asList(args));
		return 0;
	}
} 

In the example, the public **no-args constructor** calls the super constructor with **true**, requesting logging to be enabled and providing the string **minimal-node** to be used as way of identifying this application in log messages.

In the **initialize** method, we pass the method's arguments to the superclass to be stored in superclass fields. The superclass provides a logging method we call to print that information.

### Configuration

In the configuration file, we add nodes to a simulation, using the following syntax; each in a separate line:

`node` `<id>` `<interfaces>` `<control-class>` `<application-class>` `<arg1>` ... `<argn>`

where:

* `<id>` is the integer id of the node (starting in 0 and following a strict increasing order);
* `<interfaces>` is the number of network interfaces attached to the node;
* `<control-class>` is the class providing the default control logic for the node;
* `<application-class>` is the class that implements the node application logic;
* `<arg1> ...` is a space separated list of string arguments passed to the previous class.

#### Example 
The example below shows a configuration file for a simulation that will
have 2 nodes. [EndSystemControl]() is a sample control-class already provided by the CNSS simulator library. 

In [ ]:
%%writefile minimal-node.config.txt

node 0 0 cnss.lib.EndSystemControl MinimalNode arg1 arg2 
node 1 0 cnss.lib.EndSystemControl MinimalNode arg3 arg4


#### Execution

The following Unix commands compile and execute the simulator.

In [ ]:
%%bash

# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java

javac -cp .:cnss-classes MinimalNode.java
java -cp .:cnss-classes cnss.simulator.Simulator minimal-node.config.txt

### Periodic actions

Nodes can request an action to be executed, periodically, ie., at regular intervals, by the way of receiving periodic clock tick events. This is done by:

1. Returning a positive value (in milliseconds) in the **initialize** method;
2. Implementing the `on_clock_tick` method with the desired behavior to be executed periodically.

All events are delivered to nodes via a call to the appropriate `on_something` method. These calls from the simulator kernel (or node kernel in this case) to the user provided code are called **up calls**.


In [ ]:
%%writefile PeriodicActionNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class PeriodicActionNode extends AbstractApplicationAlgorithm {

  public PeriodicActionNode() {
      super(true, "periodic-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);
		return 1500;
	}

  public void on_clock_tick(int now) {
    super.log( now, "on_clock_tick");      
  }

}

In the updated example above, we request that method `on_clock_tick` to be called every 1500 milliseconds, as measured in simulation virtual time (not realtime).

The updated configuration below, adds a `stop` parameter line to terminate the simulation after 10000 milliseconds of virtual time.

In [ ]:
%%writefile periodic-action.config.txt

node 0 0 cnss.lib.EndSystemControl PeriodicActionNode 
node 1 0 cnss.lib.EndSystemControl PeriodicActionNode


parameter stop 10000

***Compilation and Execution***

In [ ]:
%%bash
# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes PeriodicActionNode.java
java -cp .:cnss-classes cnss.simulator.Simulator periodic-action.config.txt

### Timeout Events

Application nodes can also schedule an operation to execute once in the future by setting a timeout. This is achieved via the `set_timeout( int )` method in the [Node](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/Node.java) class.

When the timeout deadline expires, the `on_timeout()` method the node implements to satisfy the [ApplicationAlgorithm](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/ApplicationAlgorithm.java) interface is called by the simulator. 

Note: Timeout events are automatically cancelled in some cases that will be discussed later on future assignments...

#### Example

In [ ]:
%%writefile TimeoutHandlingNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class TimeoutHandlingNode extends AbstractApplicationAlgorithm {

  public TimeoutHandlingNode() {
      super(true, "timeout-handling-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);
    self.set_timeout(1000 + node_id);
		return 0;
	}

  public void on_timeout(int now) {
    super.log( now, "on_timeout");      
  }

}

***Configuration***

In [ ]:
%%writefile timeout-handling.config.txt

node 0 0 cnss.lib.EndSystemControl TimeoutHandlingNode 
node 1 0 cnss.lib.EndSystemControl TimeoutHandlingNode


parameter stop 10000

***Execution***

In [ ]:
%%bash
# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes TimeoutHandlingNode.java
java -cp .:cnss-classes cnss.simulator.Simulator timeout-handling.config.txt

***Question:*** The timeout events are not executed in the same instant in the two nodes. Why is that?

---

## CNSS Networking

The goal of networking is to exchange information among nodes via network packets.

In CNSS, **data packets** are modelled by the [DataPacket](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/DataPacket.java) class, and are created by providing the destination node **id**, and the message payload or contents as an array of bytes.

In the example below, we define a node that creates and sends empty data packets to **itself** every 500 ms.

In [ ]:
%%writefile SelfSenderNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class SelfSenderNode extends AbstractApplicationAlgorithm {

  public SelfSenderNode() {
      super(true, "selfsender-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);

		return 500;
	}

  public void on_clock_tick(int now) {
      self.send(self.createDataPacket( super.nodeId, new byte[1]));
  }

  public void on_receive( int now, DataPacket p ) {
    log( now, "got: " + p); 
  }
} 

***Configuration***

In [ ]:
%%writefile self-sender.config.txt

node 0 1 cnss.lib.EndSystemControl SelfSenderNode 

parameter stop 8000

***Compilation and Execution***

In [ ]:
%%bash
# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes SelfSenderNode.java
java -cp .:cnss-classes cnss.simulator.Simulator self-sender.config.txt

One interesting aspect to note is the fact that all packets are delivered to their destination after some time. A packet sent by a node to itself is delivered exactly 1 ms later since in CNSS virtual time advances in milliseconds (which are the smaller time frame CNSS supports).

### Network Links

To simulate a network we require network links. Links inter-connect nodes by linking two **interfaces**.


### Configuration

In the configuration file, we add links to the simulation using the following syntax; each in a separate line:

`link` `<id1>`.`<interface1>` `<id2>`.`<interface2>` `<bandwidth>` `<latency>` `<error_rate>` `<jitter>`

where:

* `<id1>` and `<id2>` identify the end nodes of the link;
* `<interface1>` and `<interface2>` identify the two interfaces linked together by the link; 
* `<bandwidth>` is the transmission rate of the link in bps;
* `<latency>` is the latency of the link in ms;
* `<error_rate>` is the error rate in percent;
* `<jitter>` is the jitter rate of the link.

<br>

#### Example 
The example below shows a configuration file for a simulation that will
have 2 nodes, connected by a single link.

In [ ]:
%%writefile one-link.config.txt

node 0 1 cnss.lib.EndSystemControl SenderNode 
node 1 1 cnss.lib.EndSystemControl ReceiverNode

link 0.0 1.0 1000000 125 0 0

parameter stop 10000

In [ ]:
%%writefile SenderNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class SenderNode extends AbstractApplicationAlgorithm {

  public SenderNode() {
      super(true, "sender-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);
		return 1000;
	}

  public void on_clock_tick(int now) {
      self.send( self.createDataPacket( 1, new byte[0]));
  }
} 

***Question:*** In the above code, what is the meaning of ***1*** in the `createDataPacket` call? What would happen if instead ***0*** was used?

In [ ]:
%%writefile ReceiverNode.java

import java.util.Arrays;
import cnss.simulator.*;
import cnss.lib.*;

public class ReceiverNode extends AbstractApplicationAlgorithm {

  public ReceiverNode() {
      super(true, "receiver-node");
  }

  public int initialise(int now, int node_id, Node self, String[] args) {
    super.initialise(now, node_id, self, args);
		return 0;
	}

  public void on_receive( int now, DataPacket p ) {
    log( now, "got: " + p); 
  }
} 

***Compilation and Execution***

In [ ]:
%%bash
# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes SenderNode.java
javac -cp .:cnss-classes ReceiverNode.java
java -cp .:cnss-classes cnss.simulator.Simulator one-link.config.txt

### Switches

CNSS can simulate networks where some nodes are packet switches, i.e. special nodes that know how to forward packets between links, so that
network packets can reach their destination nodes.

The base CNSS code already provides classes to instantiate a basic rswitch node. Such nodes do not require application-level actions, and can reply on the [EmptyApp](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/library/EmptyApp.java) class. Whereas, the [FloodingSwitch](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/lib/FloodingSwitch.java) class implements the default packet forwarding protocol.

It is interesting to note now that in CNSS all nodes can execute a control algorithm as well as an application algorithm. Switch only nodes execute a packet forwarding algorithm (the control algorithm) and an empty application algorithm, while end systems or application only nodes, execute an elementary control algorithm made available in the lib code base of CNSS. This simple control algorithm allows the application node to send and receive packets using its single link.

#### Example configuration

To exemplify a network where 2 nodes connected via an intermediary switch, we add a third node to the simulation configuration file.

Note that that the **switch** has 2 interfaces, each at the end of one of the two links.

In [ ]:
%%writefile two-links.config.txt

node 0 1 cnss.lib.EndSystemControl SenderNode 
node 1 1 cnss.lib.EndSystemControl ReceiverNode
node 2 2 cnss.lib.FloodingSwitch cnss.lib.EmptyApp

link 0.0 2.0 1000000 125 0 0
link 2.1 1.0 1000000 500 0 0

parameter stop 10000

***Compilation and Execution***

In [ ]:
%%bash
# Fetch the CNSS repository and compile it
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java


javac -cp .:cnss-classes SenderNode.java
javac -cp .:cnss-classes ReceiverNode.java
java -cp .:cnss-classes cnss.simulator.Simulator two-links.config.txt

Before closing this short introduction to CNSS you should look carrefully at the log of the above simulation and try to explain how CNSS can be used to compute packets end-to-end transit time.

# Exercise 

Up to this point, we have covered the main concepts that CNSS simulations will revolve around. 

1) We know how to code Nodes that can execute operations in due time or in response to the arrival of network packtets. 

2) How to specify a configuration file that defines which nodes will be simulation, as well as, and the network links that connect them together. 

3) Compile and run the simulator and see its output results.

The challenge for this exercise is to use the CNSS simulator to find the **round-trip-time** (RTT)[https://en.wikipedia.org/wiki/Round-trip_delay] between two nodes. 

This can be achieved by measuring the virtual simulation time that a small message takes to go from a **node A** to a **node B** and back (to A).

The two requirements of the solution are:

1. The RTT should be measured/repeated every 5 seconds for the duration of the simulation;
2. The sender node should accept a single argument corresponding to the total size in bytes of the payload of the data packet that will be used to determine the RTT;
3. The solution should be general and produce the correct result with different configurations of network links.

The payload of the message used to measure the RTT is up to you; for instance, it can be all zeroes or include any information you see fit, as long as it does not exceed the size specified. 

As a sugestion, you might want to consider including the time the message was sent/created as part of the packet payload...

Moreover, to address this challenge, you can modify the ***SenderNode*** and ***ReceiverNode*** classes provided earlier.
